# Week 17, Lecture 01 CodeAlong
- Coefficients & Feature Importance

## Lesson Objectives

- By the end of this lesson, students will be able to:
    - Extract feature names from sklearn v1.1 objects
    - Extract and visualize coefficients
    - Save models to a joblib file


### The Data

Data comes from the World Health Organization.  It describes demographic, health, and economic data from countries around the world between 2000 and 2015. 

Each row is one country during one year.

> Task Inspired by: https://medium.com/@shanzehhaji/using-a-linear-regression-model-to-predict-life-expectancy-de3aef66ac21

- Kaggle Dataset on Life Expectancy:
    - https://www.kaggle.com/datasets/kumarajarshi/life-expectancy-who

In [ ]:
## Our standard imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as miss

## Preprocessing tools
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

## Models & evaluation metrics
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import joblib

## setting random state for reproducibility
SEED = 321
np.random.seed(SEED)
## Matplotlib style
fav_style = ('ggplot','tableau-colorblind10')
fav_context  ={'context':'notebook', 'font_scale':1.1}
plt.style.use(fav_style)
sns.set_context(**fav_context)
plt.rcParams['savefig.transparent'] = False
plt.rcParams['savefig.bbox'] = 'tight'

In [ ]:
## Importing Custom Functions
%load_ext autoreload
%autoreload 2
from CODE import data_enrichment as de

In [ ]:
import pandas as pd
df = pd.read_csv("../Data/Life Expectancy Data.csv")
df.info()
df.head(3)

In [ ]:
# clean extra spaces
df.columns = df.columns.str.strip()
df

In [ ]:
pd.set_option('display.max_columns',0)
df

## EDA

In [ ]:
df.isna().sum()

In [ ]:
miss.matrix(df)

> Can't have null values for the target!

In [ ]:
# drop null values ONLY FROM TARGET
df = df.dropna(subset=['Life expectancy'])

In [ ]:
df.describe()

In [ ]:
target = 'Life expectancy'

grid_spec = {'height_ratios':[0.8,0.2]}
fig, axes = plt.subplots(nrows=2, figsize=(6,5), gridspec_kw=grid_spec)

sns.histplot(data=df, x=target,ax=axes[0])
sns.boxplot(data=df, x=target, ax=axes[1]);

## Preprocessing (with Sklearn v1.1+)

In [ ]:
# Run the following command on your local computer to check the version of sklearn
import sklearn
!python --version
print(f"sklearn version: {sklearn.__version__}")

In [ ]:
# ### Train Test Split
## Make x and y variables
target = "Life expectancy"
drop_feats = []

y = df[target].copy()
X = df.drop(columns=[target, *drop_feats]).copy()

## train-test-split with random state for reproducibility
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=SEED)
X_train.head(3)

In [ ]:
## Make numeric preprocessing pipeline
num_sel = make_column_selector(dtype_include='number')
num_pipe = make_pipeline(SimpleImputer(strategy='mean'))
num_pipe

<center> <font color='red' size=5>Notice We Are Not Scaling!!! </font>
    
   **<center> Q: Why not? </center>**

In [ ]:
## Make categorical preprocessing pipeline
## Drop one of the binary columns after OHE to reduce multicollinearity
cat_sel = make_column_selector(dtype_include='object')
cat_pipe = make_pipeline(SimpleImputer(strategy='constant',
                                       fill_value='MISSING'),
                         OneHotEncoder(handle_unknown='ignore', sparse=False,
                                      drop='if_binary'))

In [ ]:
## make the preprocessing column transformer
preprocessor = make_column_transformer((num_pipe, num_sel),
                                       (cat_pipe,cat_sel),)
preprocessor

### Get Features Names + Verbose Feature Names Out

In [ ]:
## make the preprocessing column transformer WITH CORRECT ARGS!
preprocessor = make_column_transformer((num_pipe, num_sel),
                                       (cat_pipe,cat_sel),
                                      verbose_feature_names_out=False)
preprocessor.fit(X_train)
X_train_df = pd.DataFrame(preprocessor.transform(X_train),
                          columns=preprocessor.get_feature_names_out(),
                         index=X_train.index)
X_test_df = pd.DataFrame(preprocessor.transform(X_test),
                          columns=preprocessor.get_feature_names_out(),
                         index=X_test.index)
display(X_train_df)


## Let's make a <span style="color:red"> SCALED </span> version as well to see how the coefficients are different.

In [ ]:
## make the preprocessing column transformer WITH CORRECT ARGS!
scaling_preprocessor = make_pipeline(preprocessor, StandardScaler())
scaling_preprocessor.fit(X_train)
X_train_scaled_df = pd.DataFrame(scaling_preprocessor.transform(X_train),
                          columns=scaling_preprocessor.get_feature_names_out(),
                         index=X_train.index)
X_test_scaled_df = pd.DataFrame(scaling_preprocessor.transform(X_test),
                          columns=scaling_preprocessor.get_feature_names_out(),
                         index=X_test.index)
display(X_train_scaled_df)

# Modeling - Linear Regression

In [ ]:
def evaluate_regression(model, X_train,y_train, X_test, y_test,for_slides=True): 
    """Evaluates a scikit learn regression model using r-squared and RMSE
    FOR SLIDES VERS DOES MULTIPLE PRINT STATEMENTS FOR VERTICAL DISPLAY OF INFO"""
    
    ## Training Data
    y_pred_train = model.predict(X_train)
    r2_train = metrics.r2_score(y_train, y_pred_train)
    rmse_train = metrics.mean_squared_error(y_train, y_pred_train, 
                                            squared=False)
    mae_train = metrics.mean_absolute_error(y_train, y_pred_train)
    

    ## Test Data
    y_pred_test = model.predict(X_test)
    r2_test = metrics.r2_score(y_test, y_pred_test)
    rmse_test = metrics.mean_squared_error(y_test, y_pred_test, 
                                            squared=False)
    mae_test = metrics.mean_absolute_error(y_test, y_pred_test)
    
    if for_slides:
        df_version =[['Split','R^2','MAE','RMSE']]
        df_version.append(['Train',r2_train, mae_train, rmse_train])
        df_version.append(['Test',r2_test, mae_test, rmse_test])
        df_results = pd.DataFrame(df_version[1:], columns=df_version[0])
        df_results = df_results.round(2)
        display(df_results.style.hide(axis='index').format(precision=2, thousands=','))
        
    else: 
        print(f"Training Data:\tR^2 = {r2_train:,.2f}\tRMSE = {rmse_train:,.2f}\tMAE = {mae_train:,.2f}")
        print(f"Test Data:\tR^2 = {r2_test:,.2f}\tRMSE = {rmse_test:,.2f}\tMAE = {mae_test:,.2f}")



### Linear Model Assumptions

**Linearity:**
That the input features have a linear relationship with the target.

**Independence of Features:** (AKA Little-to-No Multicollinearity)
That the features are not strongly related to other features.

**Normality:**
The model's residuals are approximately normally distributed.

**Homoscedasticity:**
The model residuals have equal variance across all predictions.

## Model 1: Baseline LinReg

In [ ]:
from sklearn.linear_model import LinearRegression
lin_reg= LinearRegression()
lin_reg.fit(X_train_df,y_train)
evaluate_regression(lin_reg, X_train_df, y_train, 
                    X_test_df, y_test)

In [ ]:
de.plot_residuals(lin_reg, X_test_df,y_test)

### Extracting and Visualizing Coefficients

#### Extracting Coefficients

In [ ]:
# access the .coef_ 



In [ ]:
# Intercept



In [ ]:
## Saving the coefficients



### def `get_coefficients`

In [ ]:
## formatting numbers to not use , thousands sep, and 4 digits floats
pd.set_option('display.float_format', lambda x: f"{x:,.4f}")


In [ ]:
# Define get_coefficients function to extract LinReg coefficients
def get_coefficients(lin_reg):
    coefs = lin_reg.coef_
    coefs = pd.Series(coefs, index=lin_reg.feature_names_in_)
    return coefs

## Scaled vs Unscaled Data

* **Scaled**: absolute value of coefficients show importance of features, or change in target due to change in one standard deviation of feature


* **Unscaled**: show impact of change of one unit for each feature on target.


### Let's see what coefficients look like for <span style="color:red"> SCALED </SPAN> data.

In [ ]:
# Fit a new model on a scaled version of the data
lin_reg_scaled = LinearRegression().fit(X_train_scaled_df, y_train)

# Examine new coefficients
get_coefficients(lin_reg_scaled).sort_values(ascending=False)

### Now lets try <span style="color:blue"> UNSCALED </span> data coefficients

In [ ]:
get_coefficients(lin_reg)

## Interpreting Coefficients

> Q: What does this coefficient tell us?

> **Target**: National Life expectancy

> **Coefficient**: Status_Developing = -10.4206

> <font color='green' size=3>**What does this coefficient tell us about the relationship between the feature and the target?**

## Visualizing Coefficients

#### **Q: How can we handle this many coefficients in our viz?**

- Some options include:
    - Only plot most the N most positive and N most negative.
    - Separate out OHE countries into a separate graph.
    
    
- Let's try the second option: separating the OHE countries.

In [ ]:
## Plot Coefficients



In [ ]:
## Plot country ceoffs

def plot_coefficients(coefs):
    ax = coefs.sort_values().plot(kind='barh', figsize=(8,26))
    ax.axvline(0, color='k')
    ax.set(xlabel='Life Expectancy', title="Coefficients - Countries")
    plt.show();

In [ ]:
## Get a list of all of the Country columns



In [ ]:
## Plot country ceoffs



In [ ]:
## Examine coeffients that are not countries



In [ ]:
# Check stats of features (not countries) to compare to coefficients



In [ ]:
## Plot everything but countries


> ***Q1: What do we notice about our non-country coefficients? Is there anything odd that would be difficult to a stakeholder?***

> ***Q2: What does that intercept represent?***

> ***Q3: What would it mean if we did not use an intercept? (fit_intercept=False)***

## Other things to try to meet assumptions or get better coefficients.

1. Remove some multicollinear columns, like countries.
2. No intercept `LinearRegression(fit_intercept=False)`
3. Remove redundant categories with OneHotEncoder `OneHotEncoder(drop='first')` _(we already did this!)_ or `OneHotEncoder(drop='if_binary)`.

Let's try dropping countries:

In [ ]:
lin_reg_drop = LinearRegression()
X_train_df = X_train_df.drop(columns=country_feats)
X_test_df = X_test_df.drop(columns=country_feats)

lin_reg_drop.fit(X_train_df, y_train)


In [ ]:
evaluate_regression(lin_reg_drop, X_train_df, y_train, X_test_df, y_test)

In [ ]:
de.plot_residuals(lin_reg_drop, X_test_df,y_test)

### Explore new coefficients

In [ ]:
#Save and display new coefficients (after dropping countries)


In [ ]:
# describe new DF to compare coefficients
X_train_df.describe()

Why does GDP per Capita seem to have no effect on Life Expectancy?

In [ ]:
# Median GDP



The median GDP per Capita in the dataset is $3,184 US.

In [ ]:
# GDP Coefficient



GDP per Capita adds 1 year of life for every 44,500 US Dollars of GDP per Capita.  

In [ ]:
## Effect of median GDP on target



Countries with the median average GDP per Capita add about 2 months of expected life from this feature.

In [ ]:
## Max GDP



In [ ]:
# Effect: Max GDP times coefficient



The country with the highest GDP per Capita adds 5 years to life expectancy with this feature.

What about Population?

In [ ]:
# Population coefficient



In [ ]:
# Population median



In [ ]:
# effect of coefficient on median population country



The countries with a median average population add less than a day to their life expectancy.

However...

In [ ]:
# Max population



In [ ]:
# Effect of max population on target



The country with the highest population adds 5 years as a result of the population size!

## Feature Importance

**Feature importance does <span style="color:red"> NOT </span> describe the relationship between features targets.**

**It only describes what the model is focusing on to make its predictions.**

**Think of the results as percentage weights on the features for how important they are.**

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rf_reg = RandomForestRegressor()
rf_reg.fit(X_train_df,y_train)
evaluate_regression(rf_reg, X_train_df, y_train, 
                    X_test_df, y_test)

> Using the models .feature_names_in_

In [ ]:
# Extract Feature Importances



In [ ]:
# create a function to extract importances



In [ ]:
# Plot Importances




> **Q1:** What do these numbers mean?

> **Q2:** What are the top 5 most important features?

## Using joblib to Save our Model, Data, and Objects

In [ ]:
X_train.head()

In [ ]:
evaluate_regression(lin_reg_drop, X_train_df, y_train, X_test_df, y_test)

In [ ]:
evaluate_regression(rf_reg, X_train_df, y_train, X_test_df, y_test)

In [ ]:
import joblib, os

## creating a dictionary of all of the variables to save for later



In [ ]:
# Create the folder to save it in



In [ ]:
# Save the models, data, and preprocessor



In [ ]:
# try Loading again to make sure it works.



> We will continue working with this task and these models next class!

# *Teaser* Shap (For Regression)

In [ ]:
# Import and init shap
import shap
shap.initjs()

In [ ]:
# Take a sample of the training data
X_shap = shap.sample(X_train_df,nsamples = 500,random_state=SEED)
y_shap = y_train.loc[X_shap.index]

# Instantiate a Model Explainer with the model
explainer = shap.Explainer(rf_reg)

## Get shap values form the explainer
shap_values = explainer(X_shap,y_shap)

In [ ]:
shap.summary_plot(shap_values, features = X_shap)